# Interpolations
Fitting a line/curve that intercept every data points.
Discussed here:
- Linear interpolation
- Cubic Spline interpolation
- .

## Linear Interpolation

In [12]:
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
import numpy as np
import inspect
plt.style.use('seaborn-poster')

C:\Users\RAYHAN EGAR\AppData\Local\Temp\ipykernel_9484\877556802.py:5: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-poster')


In [13]:
def linear_interpolation(x, y, X):
    xArr = np.array(x).astype(float)
    yArr = np.array(y).astype(float)
    XArr = np.array(X).astype(float)
    n = xArr.size
    f = [None] * (n-1)
    for i in range(0, n-1):
        xleft = xArr[i]
        yleft = yArr[i]
        xright = xArr[i+1]
        yright = yArr[i+1]
        Xcurr = XArr[i]
        f[i] = yleft + (yright - yleft) / (xright - xleft) * (Xcurr - xleft)
    return f

In [14]:
f_test = linear_interpolation([1, 2, 3, 4], [14, 6, 11, 11], [1.81, 2.6, 3.65])
f_bench = interp1d([1, 2, 3, 4], [14, 6, 11, 11])
y_bench = f_bench([1.81, 2.6, 3.65])
print(f_test)
print(y_bench)

[7.52, 9.0, 11.0]
[ 7.52  9.   11.  ]


## Polynomial Interpolation: Lagrange's Method
$$P_n(x) = \sum _{i=0} ^n y_i l_i(x)$$
$$l_i(x) = \Pi _{j=0, j /= i}^n \frac{x - x_i}{x_i - x_j} $$

In [29]:
def lagrange_interpolation(xData, yData, X):
    xArr = np.array(xData).astype(float)
    yArr = np.array(yData).astype(float)
    n = xArr.size

    def P(x):
        result = 0
        for i in range(n):
            result += yArr[i] * L(i, x)
        return result
    
    def L(i, x):
        numerator = np.prod([x - xArr[j] for j in range(n) if j != i])
        denominator = np.prod([xArr[i] - xArr[j] for j in range(n) if j != i])
        return numerator / denominator
    
    print(f"P({X}) = ", P(X))
    return P(X)

In [30]:
x_lagrange_test = [0, 1, 2]
y_lagrange_test = [1, 3, 2]
X_lagrange_test = 1.5

lagrange_interpolation(x_lagrange_test, y_lagrange_test, X_lagrange_test)

P(1.5) =  2.875


2.875

## Polynomial Interpolation: Newton Interpolation

In [33]:
def newton_coefficient(xData, yData):
    xArr = np.array(xData).astype(float)
    yArr = np.array(yData).astype(float)
    n = xArr.size
    a = yArr.copy()

    for i in range(1, n):
        currA = a[i-1] 
        currX = xArr[i-1]

        for k in range(i, n):
            a[k] = (a[k] - a[k-1])/(xData[k] - xData[k-1])
    return a

def newton_interpolation(a, xData, X):
    n = len(xData) - 1
    p = a[n] # init with first coefficient in a, x degree of 0

    for k in range(1, n+1):
        p = a[n-k] + (X - xData[n-k])*p 
        # a[n-k] will be used as multiplication coefficient in the next iteration, get it?
    
    print(f"P{n}({X}) = {p}")
    return p

In [34]:
x_newton_test = [0, 1, 2]
y_newton_test = [1, 3, 2]
a_coefficients = newton_coefficient(x_newton_test, y_newton_test)
X_newton_test = 1.5
result = newton_interpolation(a_coefficients, x_newton_test, X_newton_test)
print(result)

P2(1.5) = 2.5
2.5


## Cubic Spline Interpolation

- For n+1 knots, notation $f_{i, i+1}(x)$ denotes cubic polynomial that spans the segment between knots $i$ and $i+1$.
- $k_i$ denoting the second derivative of the spline at knot $i$. $$f_{i-1, i}^" (x_i) = f_{i, i+1}^" (x_i) = k_i$$
- each $k$ is unknown except for $$ k_0 = k_n = 0$$

### Roadmap
Start with $f_{i, i+1}^"(x) = k_il_i(x) + k_{i+1}l_{i+1}(x)$
- $l_i(x) = \frac{x - x_{i+1}}{x_i - x_{i+1}}$
- $l_{i+1}(x) = \frac{x - x_i}{x_{i+1} - x_i}$

Therefore
$$f_{i, i+1}^"(x) = \frac{k_i(x - x_{i+1}) + k_{i+1}(x - x_i)}{x_i - x_{i+1}}$$
$$f_{i, i+1}(x) = \frac{k_i(x - x_{i+1})^3 + k_{i+1}(x - x_i)^3}{6(x_i - x_{i+1})} + A(x-x_{i+1}) - B(x - x_{i})$$
$$f_{i, i+1}(x) = \frac{k_i(x - x_{i+1})^3 + k_{i+1}(x - x_i)^3}{6(x_i - x_{i+1})} + (A-B)x + (-Ax_{i+1} + Bx_i)$$

### Coefficients
- $C = A-B$ (from above eq.)
- $D = -Ax_{i+1} + Bx_i$

### Equations used
$$f_{i, i+1}(x) = \frac{k_i}{6} [\frac{(x-x_{i+1})^3}{x_i - x_{i+1}} - (x - x_{i+1}(x_i - x_{i+1}))]
- \frac{k_{i+1}}{6} [\frac{(x-x_{i})^3}{x_i - x_{i+1}} - (x - x_{i}(x_i - x_{i+1}))] 
+ \frac{y_i (x-x_{i+1}) //- y_{i+1}(x-x_i)}{x_i - x_{i+1}}

In [17]:
def gauss_elimination(a, b):

    n = len(b)
    x = np.zeros(n).astype(float)

    # iterates pivot row
    # pivot element: main diagonal element, hence why 'j' is the pivot
    for j in range(n):

        # iterates for each modified row (pivot + 1)
        for i in range(j + 1, n):
                ratio = a[i, j] / a[j, j]
                a[i, j:n] = a[i, j:n] - ratio * a[j, j:n]
                b[i] = b[i] - ratio * b[j]

    # back-substitution phase
    for i in range(n-1, -1, -1):
        sum_term = np.dot(a[i, i+1:n], x[i+1:n])
        x[i] = (b[i] - sum_term) / a[i, i]

    return x

In [18]:
def lu_decomposition(a, b):
    n = len(a)
    u = a.copy()
    l = np.eye(n)

    # LU factorization of A
    for j in range(n):
        for i in range(j+1, n):
            l[i, j] = u[i, j] / u[j, j]
            u[i, j:n] = u[i, j:n] - l[i, j] * u[j, j:n]
    
    # ly = b, find y using gauss_elimination(l, b)
    y = gauss_elimination(l, b)

    # ux = y, find x using gauss_elimination(u, y)
    x = gauss_elimination(u, y)

    return x

In [35]:
def LUdecomp3(c,d,e):
    n = len(d)
    for k in range(1,n):
        lam = c[k-1]/d[k-1]
        d[k] = d[k] - lam*e[k-1]
        c[k-1] = lam
    return c,d,e

def LUsolve3(c,d,e,b):
    n = len(d)
    for k in range(1,n):
        b[k] = b[k] - c[k-1]*b[k-1]
        b[n-1] = b[n-1]/d[n-1]
    for k in range(n-2,-1,-1):
        b[k] = (b[k] - e[k]*b[k+1])/d[k]
    return b
        

In [36]:
def curvatures(xData, yData):
    n = len(xData) - 1
    c = np.zeros(n)
    d = np.ones(n+1)
    e = np.zeros(n)
    k = np.zeros(n+1)

    # Height diff. between segment
    c[0:n-1] = xData[0:n-1] - xData[1:n]

    # twice height diff. 
    d[1:n] = 2.0 * (xData[0:n-1] - xData[2:n+1])

    # height diff. starting from the second segment
    e[1:n] = xData[1:n] - xData[2:n+1]

    # second derivatives 'k' 
    k[1:n] = 6.0 * (yData[0:n-1] - yData[1:n]) / (xData[0:n-1] - xData[1:n]) - 6.0 * (yData[1:n] - yData[2:n+1]) / (xData[1:n] - xData[2:n+1])

    p,q,r = LUdecomp3(c,d,e)
    res = LUsolve3(p,q,r,k)
    return res


In [39]:
def cubic_spline_interpolation(xData, yData, k, x):

    def findSegment(xData, x):
        iLeft = 0
        iRight = len(xData)-1
        while 1:
            if (iRight - iLeft) <= 1:
                return iLeft
            i = (iLeft + iRight) // 2
            if x < xData[i]:
                iRight = i
            else:
                iLeft = i
    
    i = findSegment(xData, x)
    h = xData[i] - xData[i+1] #calculates height diff. for segment i
    
    y = ((x - xData[i+1])**3/h - (x - xData[i+1])*h)*k[i]/6.0 \
        - ((x - xData[i])**3/h - (x - xData[i])*h)*k[i+1]/6.0 \
        + (yData[i]*(x - xData[i+1]) \
        - yData[i+1]*(x - xData[i]))/h
    
    return y